In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

## Limpieza de train

In [3]:
train = pd.read_csv("02_prepare_data/output/train.csv")
test =  pd.read_csv("02_prepare_data/output/test.csv")

le = LabelEncoder()
train['NObeyesdad'] = le.fit_transform(train['NObeyesdad'])
list(le.classes_)


train_x = train.drop(['NObeyesdad','id' ], axis = 1)
train_x.insert(train_x.columns.get_loc('CALC_Frequently'), 'CALC_Always', 0)
train_y = train['NObeyesdad']

test_x = test

In [4]:
model = xgb.XGBClassifier(objecztive='multi:softprob')

## Param grid Model 4

In [5]:
param_grid = {
    'max_depth': [5,6, 7],
    'learning_rate': [0.1], #
    'subsample': [1], 
    'colsample_bytree': [0.5],
    'n_estimators' : [300,500]
    }

In [ ]:
boost_grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5,scoring = 'accuracy', 
                         verbose = 0, return_train_score = True)

boost_grid.fit(train_x, train_y)

In [7]:
results = pd.DataFrame(boost_grid.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.iloc[:3]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,5.175293,0.759539,0.093167,0.015452,0.5,0.1,5,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908806,0.002847,1,0.961941,0.962243,0.962363,0.963509,0.963269,0.962665,0.000612
2,5.746084,0.172865,0.092633,0.013147,0.5,0.1,6,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908566,0.003481,2,0.979465,0.979104,0.978562,0.980490,0.981153,0.979755,0.000941
3,9.716999,0.162553,0.147960,0.026381,0.5,0.1,6,500,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.907409,0.002913,3,0.991690,0.991931,0.991991,0.992654,0.992593,0.992172,0.000383


In [8]:
id = test_x['id']
test_x = test_x.drop(['id'], axis=1)
prediction = boost_grid.predict(test_x)
prediction = le.inverse_transform(prediction)
submission = pd.DataFrame({'id':id,'NObeyesdad':prediction})

In [9]:
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [10]:
submission.to_csv("04_submissions/output/xgboost_4.csv", index=False)

## Param grid Model 3

In [11]:
param_grid = {
    'max_depth': [5,6],
    'learning_rate': [0.1, 0.2, 0.3], #
    'subsample': [1], 
    'colsample_bytree': [0.5],
    'n_estimators' : [300,400]
    }

In [ ]:
boost_grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5,scoring = 'accuracy', 
                         verbose = 0, return_train_score = True)

boost_grid.fit(train_x, train_y)

In [13]:
results = pd.DataFrame(boost_grid.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.iloc[:3]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,4.590701,0.296134,0.071005,0.008417,0.5,0.1,5,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908806,0.002847,1,0.961941,0.962243,0.962363,0.963509,0.963269,0.962665,0.000612
1,6.554492,0.387007,0.089663,0.016199,0.5,0.1,5,400,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908758,0.003947,2,0.971998,0.973082,0.972962,0.974469,0.974228,0.973348,0.000902
2,6.546445,0.671111,0.097653,0.008091,0.5,0.1,6,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908566,0.003481,3,0.979465,0.979104,0.978562,0.980490,0.981153,0.979755,0.000941


In [14]:
id = test_x['id']
test_x = test_x.drop(['id'], axis=1)
prediction = boost_grid.predict(test_x)
prediction = le.inverse_transform(prediction)
submission = pd.DataFrame({'id':id,'NObeyesdad':prediction})

In [15]:
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [16]:
submission.to_csv("04_submissions/output/xgboost_3.csv", index=False)

## Param grid Model 2

In [4]:
param_grid = {
    'max_depth': [5,6],
    'learning_rate': [0.01, 0.01,0.1], #
    'subsample': [1], 
    'colsample_bytree': [0.5],
    'n_estimators' : [300]
    }

In [5]:
boost_grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5,scoring = 'accuracy', 
                         verbose = 0, return_train_score = True)

boost_grid.fit(train_x, train_y)

c:\Users\jorge_j24fcle\OneDrive\Documentos\3. coding\R\kaggle_playground4_2\kaggle_env\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:41:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objecztive" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\jorge_j24fcle\OneDrive\Documentos\3. coding\R\kaggle_playground4_2\kaggle_env\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:41:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objecztive" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\jorge_j24fcle\OneDrive\Documentos\3. coding\R\kaggle_playground4_2\kaggle_env\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:41:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboo

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objecztive='multi:softprob', ...),
             param_grid={'colsample_bytree': [0.5],
                         'learning_rate': [0.01, 0.01, 0.1],
                         'max_depth': [5, 6], 'n_estimators': [300],
                         'subsample': [1]},
             return_train_score=True, scoring='accuracy')

In [6]:
results = pd.DataFrame(boost_grid.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.iloc[:3]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
4,4.799436,0.265806,0.084485,0.017046,0.5,0.1,5,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908806,0.002847,1,0.961941,0.962243,0.962363,0.963509,0.963269,0.962665,0.000612
5,5.834896,0.434579,0.094224,0.011685,0.5,0.1,6,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",...,0.908566,0.003481,2,0.979465,0.979104,0.978562,0.980490,0.981153,0.979755,0.000941
1,7.168300,0.797201,0.102820,0.013450,0.5,0.01,6,300,1,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",...,0.899750,0.004520,3,0.919366,0.919366,0.915151,0.918890,0.918709,0.918297,0.001594


In [7]:
id = test_x['id']
test_x = test_x.drop(['id'], axis=1)
prediction = boost_grid.predict(test_x)
prediction = le.inverse_transform(prediction)
submission = pd.DataFrame({'id':id,'NObeyesdad':prediction})

In [9]:
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [8]:
submission.to_csv("04_submissions/output/xgboost_2.csv", index=False)

## Param grid Model 1

In [ ]:
param_grid = {
    'eta': [0.01, 0.001], #
    'max_depth': [5,6,7],
    'learning_rate': [0.1,0.2], #
    'subsample': [1], 
    'colsample_bytree': [0.5],
    'n_estimators' : [200]#
    }

In [ ]:
boost_grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5,scoring = 'accuracy', 
                         verbose = 0, return_train_score = True)

boost_grid.fit(train_x, train_y)

In [ ]:
results = pd.DataFrame(boost_grid.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.iloc[:3]

In [ ]:
id = test_x['id']
test_x = test_x.drop(['id'], axis=1)
prediction = boost_grid.predict(test_x)
prediction = le.inverse_transform(prediction)
submission = pd.DataFrame({'id':id,'NObeyesdad':prediction})

In [ ]:
submission.to_csv("04_submissions/output/xgboost_1.csv", index=False)